# RAG Rerank Retrieval Experiment

In this experiment, we will experiment with pure document retrieval without reranking, as typically found in Naive RAG systems, and compare it to the improvements achieved by utlilsing reranking. 

Reranking is a technique that can dramatically optimise our retrieval pipelines and enhance their accuracy. To learn more about how reranking works and the challenges it addresses, visit [this](https://www.notion.so/fuzzylabs/Reranking-04e3f64f27724e51875abd7eb7d97a3c#6f1682a7b243482a8d226f675816851c) notion page.

In this example notebook, we will demonstrate how to create retrieval pipelines with reranking using the [BAAI/bge-reranker-large](https://huggingface.co/BAAI/bge-reranker-large) reranker, which has comparable performance to Cohere's paid model, according to Llamaindex.

The following components are used:
- VectorDB - [Pinecone](https://www.pinecone.io/)
- Embedding Model - [jinaai/jina-embeddings-v2-base-en](https://huggingface.co/jinaai/jina-embeddings-v2-base-en)
- Reranker - [BAAI/bge-reranker-large](https://huggingface.co/BAAI/bge-reranker-large)


# Parameters
The following parameters are used in the experiment.
   
`QUERY = "Who manufactured STM32F429ZIT6U microcontroller?"`

We have chosen the above query to check for specific content within the context and to evaluate improvements after applying reranking to the retrieval results.

Specifically, we are looking for following content in the retrieved documents.

`In this chapter, many programs were developed using the NUCLEO board, provided with the STM32F429ZIT6U microcontroller. This microcontroller is manufactured by STMicroelectronics [13] using a Cortex-M4 processor designed by Arm Ltd. [14].`

In [1]:
######## Parameters #########

# Data
FILE_PATH = "data/a_beginners_guide_to_designing_embedded_system_applications_on_arm_cortex-m_microcontrollers.pdf"
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 50
COLLECTION_NAME = 'experiment_search'
BATCH_SIZE = 32

# Embedding model
EMBEDDING_MODEL = "jinaai/jina-embeddings-v2-base-en"

# Reranker
RERANKER_MODEL = 'BAAI/bge-reranker-large'

# Query
TOP_K = 3
QUERY = "Who manufactured STM32F429ZIT6U microcontroller?"

We want to utilise GPU to speed up computation time if available.


In [2]:
import torch

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using device={DEVICE}")

Using device=cpu


# Data Preprocessing
> Make sure the pdf data is present in [data](data) folder.

In this step, we preprocess the our example PDF data defined by FILE_PATH parameter. This pre-processed data will be added in the vector store.

- We use [PyPDFLoader](https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/pdf/#using-pypdf) from langchain to read the PDF.
- We use [RecursiveCharacterTextSplitter](https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/) from langchain to split the text in small chunks. The parameters `CHUNK_SIZE` and `CHUNK_OVERLAP` configure the chunk size and chunk overlap.
> This step takes about 3 mins for a 600 pages PDF document.

In [3]:
%%time

from data_pipeline import prepare_data
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)
chunked_texts = prepare_data(FILE_PATH, text_splitter)

CPU times: user 2min 42s, sys: 3.93 s, total: 2min 46s
Wall time: 3min 6s


# Embedding The Data

In this step, we will use Jina AI's embedding model to embed all of our texts and create a pandas DataFrame to associate the embeddings with their corresponding texts.

> This step is resource intensive and it's much faster on Nvidia GPU.
>
> It might take around 10-15 mins if running on CPU.
>
> It calculates the embeddings for entire dataset.

In [4]:
%%time

from embedding_model import embed_locally

text_embeddings = embed_locally(chunked_texts)

/Users/oscar/Library/Caches/pypoetry/virtualenvs/rag-retrieval-rerank-_oC8k4Cq-py3.10/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


CPU times: user 51min 30s, sys: 3min 50s, total: 55min 20s
Wall time: 15min 4s


In [5]:
import pandas as pd

data_to_store = pd.DataFrame({'embeddings': text_embeddings})
data_to_store['metadata'] = [{'texts': text} for text in chunked_texts]
data_to_store.index.name = 'id'
data_to_store.reset_index(inplace=True)
data_to_store['id'] = data_to_store['id'].astype(str)

> We include a metadata column containing the corresponding embedding texts in a dictionary because Pinecone cannot store strings directly and only allows text within the metadata.

In [6]:
data_to_store

,id,embeddings,metadata
0,0,"[-0.022770140320062637, -0.049992650747299194,...",{'texts': 'A Beginner’s Guide to Designing Em...
1,1,"[-0.04170405492186546, -0.04468979313969612, 0...",{'texts': 'A Beginner’s Guide to Designing E...
2,2,"[-0.036007110029459, -0.06197873130440712, 0.0...",{'texts': 'A Beginner’s Guide to Designing E...
3,3,"[-0.028807535767555237, -0.03928277641534805, ...",{'texts': 'Arm Education Media is an imprint o...
4,4,"[-0.012164550833404064, -0.04263245314359665, ...",{'texts': 'book. Notices Knowledge and best pr...
...,...,...,...
1630,1630,"[-0.04276715964078903, -0.048491425812244415, ...",{'texts': 'ISBN 978-1911531-16-6 Operating Sy...
1631,1631,"[-0.03923221305012703, -0.051492903381586075, ...",{'texts': 'A Beginner’s Guide to Designing Em...
1632,1632,"[-0.04426056891679764, -0.06682316213846207, 0...",{'texts': 'align with a typical twelve-week se...
1633,1633,"[-0.052978359162807465, -0.04494466632604599, ...",{'texts': 'Arm Education Media is a publishing...


# Create Vector Database with Pinecone


Now we create our vector DB to store our vectors. For this we need to get a [free Pinecone API key](https://app.pinecone.io/) — the API key can be found in the "API Keys" button found in the left navbar of the Pinecone dashboard.

In [7]:
from pinecone import Pinecone, ServerlessSpec
import time
import getpass

# initialise connection to pinecone (get API key at app.pinecone.io)
api_key = getpass.getpass()

# configure client
pc = Pinecone(api_key=api_key)

> An "Index" is a database in Pinecone.

In [8]:
index_name = "rerankers"
existing_indexes = [
    index_info["name"] for index_info in pc.list_indexes()
]

# check if index already exists (it shouldn't if this is first time)
if index_name not in existing_indexes:
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=768,  # embedding dimensionality of jina-embeddings-v2-base
        metric='cosine',
        spec=ServerlessSpec(
            cloud="aws", region="us-east-1"
        )
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

# Data Ingestion

In this step, we store the embeddings and chunked texts in the Pinecone database we just created.

In [9]:
from tqdm.auto import tqdm

batch_size = 100  # how many embeddings we create and insert at once

for i in tqdm(range(0, len(data_to_store), batch_size)):
    passed = False
    # find end of batch
    i_end = min(len(data_to_store), i+batch_size)
    # create batch
    batch = data_to_store[i:i_end]
    to_upsert = list(zip(batch["id"], batch["embeddings"], batch["metadata"]))

    # upsert to Pinecone
    index.upsert(vectors=to_upsert)

100%|██████████| 17/17 [00:11<00:00,  1.50it/s]


# Document Retrieval

Now that we have our chunked texts and their embeddings stored in the database, we can retrieve them from Pinecone by calling the query function. The steps are as follows:

1. Embed our query.
2. Call the query function on the Pinecone index object.
3. Return the text from the result.

In [15]:
def get_closest_texts(query: str, top_k: int) -> list[tuple[str, float]]:
    # encode query
    encoded_query = embed_locally([query])
    # search Pinecone index
    res = index.query(vector=encoded_query[0], top_k=top_k, include_metadata=True)
    # get doc text
    closest_texts = [(match["metadata"]['texts'], match["score"]) for match in res["matches"]]
    return closest_texts

# Retrieval With No Reranking (Naive RAG Retrieval)

This is the most common approach for retrieving semantically similar documents to the query with, these embeddings are also known as dense vector. The dense vector captures the semantic meaning of the text using the embedding model.

The query vector is matched against all the entries in the database to find closest neighbors to the search query using a distance metric, in this case, the cosine distance.

Typically, we will look for the 3 closest documents in the database. These 3 documents will be pass to a LLM to be used as contexts to generate a response.

In [45]:
three_closest_texts = get_closest_texts(query=QUERY, top_k=3)

In [46]:
pd.set_option('display.max_colwidth', None)
three_closest_texts_df = pd.DataFrame(three_closest_texts, columns=["Retrieved Document", "Score"])
three_closest_texts_df

,Retrieved Document,Score
0,"Index\n567 STM 32F429 ZIT 6U microcontroller 5, 33, 37, 38 \n stop bit 46, 64, 65, 80, 256 , 257 , 273 , 468 \n ST Zio connectors xiv, 6, 9, 37, 38, 44, 346 \n superloop 13, 519 \n synchronous communication 255\nT TCP server 456 , 457 , 459 , 460 –463 , 491 \n temperature sensor 4, 5, 87, 106 , 123 , 182 , 220 , 233 , 334 , 486 , 550 \n time management xvi, 86, 95, 99, 289 , 342 , 348 , 353 \n timers xvi, xxvii, 37, 306 , 342 , 343 , 345 , 348 –350 , 353 , 354 , \n 361 , 441 , 512 –517 , 533 , 535 , 537 , 538 , 539 \n tm structure 153 \n TO- 220 package 89\nU UART xvi, xvii, 5, 44, 61, 63, 79, 82, 84, 86, 181 , 191 , 222 , 255 , 290 , 291 , \n 306 , 350 , 420 , 423 , 447 , 455 , 461 , 465 , 468 , 544 \n USB xiv, xxvii, 4, 9, 37, 44, 45, 63, 79, 82, 88, 132 , 449 , 506 , 507 , 508 , 544 \n USB connection xiv, 44, 506 \n use cases 497 , 501 –504 , 511 , 545 , 546 , 547 , 548\nV validation 496 , 497 , 546 , 547 \n verification 81, 92, 496 , 497 , 546 \n vineyard frost prevention 123 , 124",0.824654
1,"Chapter 1 | Introduction to Embedded Systems \n37The STM32F429ZIT6U microcontroller includes a Cortex-M4 processor , as shown in Figure 1.22. It \ncan be appreciated that, beyond the processor, the microcontroller includes other peripherals such as \ncommunication cores (ethernet, USB, UART, etc.), memory, timers, and GPIO (General Purpose Input Output) ports. \nNUCLEO\n-F429ZI32F429ZIT6U\nARM7B776 VQ\nPHL 7B 7213e412000K620 Y12000\nK620 Y\n120 00K620 YDGKYD\nKMS-1 102NL1706C\nSTM32F103CBT6\ne393701GH218CHN\nST890C\nGK717\n11\n22\n33\n44\n55\n66\n77\n88\n9\n10\n11\n12\n13\n14\n15PH_0\nPD_0\nPD_1\nPG_0PH_1PF_2PA_7PF_10PF_5PF_3PC_3PC_030\n29\n28\n27\n2616\n2515\n2414\n2313\n2212\n2111\n2010\n199\n18\n17\n165V\nVIN3.3VIOREF\nGND\nGNDGND\nNCNC\nUART2_RX\nCAN1_TDCAN1_ DRADC1/7ADC1/3\nADC1/10\nADC1/13\nADC3/9\nADC3/15\nADC3/8\nADC3/5NRSTPC_8\nPC_9\nPC_10\nPC_11\nPC_12\nPD_2\nPG_2\nPG_3\nGNDPD_7\nPD_6\nPD_5\nPD_4\nPD_3\nPE_2\nPE_4\nPE_3\nPF_7\nPG_1UART2_RX\nUART2_ XT\nUART_X7TUART2_RTS\nUART2_CTSUART3_TX\nUART3_RX\nUART5_TX\nUART5_RX\nPA_3\nSPI1_MOSISPI3_SCK\nSPI2_SCK",0.808520
2,"USAR Tn\nbxCANnsmcard\nirDA\nFIFODigital\nﬁlter\nDACnAPB1 45 MHz (max)AHB1 180 MHzAHB3\nVDD = 1.8 to 3.6V\nVSS\nVCAP1, VCAP2APB2 90 MHzART\nACCEL/CACHE\nGPIO POR TtOSC32_I N\nOSC32_OUTDMA2USB\nOTG HS\n8 Streams\nFIFODMA/\nFIFO PHY\nCHROM-AR T\nDMA2DFIFO\nFIFO8 Streams\nFIFODMA1\nLCD_R[7:0], LCD_G[7:0] ,\nLCD_B[7:0], LCD_HSYNC,\nLCD_VSYNC, LCD_DE,\nLCD_CLKLCD-TFTDMA/\nFIFO\nRTC_AF1\nRTC_AF1\nRTC 50Hz\n4KB BXPSRAMLS@ VBAT\nRTC\nAWU\nBackup registerXTAL32 KHz\nLSStandby\ninterfaceIWDGXTALOSC 4-26 MHzPOR\nreset\nInt\n@ VDDA @ VDDPVDPOR/PDR BORSupply\nsupervision@ VDDVDD\nVoltage regulator\n3.3 to 1.2VPower\nmanagement\nPLL1, 2, 3RC LSRC HS@ VDDA\nReset\n& clock\ncontrol\nFigure 1.22 STM32F429ZI block diagram made using information available from [9].",0.807650


### Result
The goal here is to verify if following context is present in any of the retrieved documents.

> In this chapter, many programs were developed using the NUCLEO board, provided with the STM32F429ZIT6U microcontroller. This microcontroller is manufactured by STMicroelectronics [13] using a Cortex-M4 processor designed by Arm Ltd. [14].

If it is present then our retrieval approach was able to find the correct document from the vector store.
> It is not present in the any of retrieved documents.

# Retrieval With Reranking

To convert texts into vectors, we are essentially compressing the "meaning" of the text into n-dimensional vectors. This compression results in some loss of information.

Due to this information loss, the top three documents (for example) retrieved by a vector search may miss relevant details, which might fall below our top_k cutoff and not be returned, as demonstrated in the example above.

Reranking addresses this issue by retrieving a larger initial set of documents from the database. A reranker then reorders these documents, retaining only the most relevant ones for our LLM.

For a deeper understanding of how reranking works and the challenges it addresses, see [this](https://www.notion.so/fuzzylabs/Reranking-04e3f64f27724e51875abd7eb7d97a3c#6f1682a7b243482a8d226f675816851c) Notion page.

In [47]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation


def get_top_3_texts_from_rerank_scores(scores: list[float], closest_texts: list[str], top_k: int) -> list[str]:
    top_k_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]
    return [closest_texts[i] for i in top_k_indices]


In [48]:
# The bge reranker takes a list of list as input

twenty_five_closest_texts = get_closest_texts(query="Who manufactured STM32F429ZIT6U microcontroller?", top_k=25)
texts_to_rerank = [[QUERY,doc[0]] for doc in twenty_five_closest_texts]

### Reranker Scores

Below are the scores which represent the similarly between our query and the retrieved texts from our vector database. The higher the score, the more relevant the document is to our query.

In [51]:
scores = reranker.compute_score(texts_to_rerank)
print(scores)

TypeError: TextInputSequence must be str

### Result

The goal here is to verify if following context is present in any of the retrieved documents.

> In this chapter, many programs were developed using the NUCLEO board, provided with the STM32F429ZIT6U microcontroller. This microcontroller is manufactured by STMicroelectronics [13] using a Cortex-M4 processor designed by Arm Ltd. [14].

If it is present then our retrieval approach was able to find the correct document from the vector store.

> It is present in the second row of Retrieved text in the first result as shown below.

In [ ]:
get_top_3_texts_from_rerank_scores(scores, twenty_five_closest_texts, top_k=3)

We can also visualise the relevance score for all 25 texts we have retrieved.

The reranker is optimised based cross-entropy loss, so the relevance score is not bounded to a specific range.

In [ ]:
scores_for_all_25_documents = pd.DataFrame({"Received Texts": twenty_five_closest_texts, "Score": scores})

In [ ]:
scores_for_all_25_documents.sort_values(by=["Score"], ascending=False)

From the score table above, we can observe that the top result (2nd row, index 0) returned by cosine distance is not as relevant compared to the first row (index 5). However, without reranking, the text at index 5 did not even make it into the top 3 when we only performed cosine distance retrieval directly from the vector database.